## Context
The Maven Market dataset comprises a global supermarket chain with hundreds of stores across the US, Mexico, and Canada. The dataset includes information on products, customers, transactions, and stores from 1997 to 1998.

## Business Goal
Your task is to deliver a presentation that provides an overview of the supermarket chain's operations. The presentation should cover key topics such as a business overview, customer demographics and behaviors, and 2-3 actionable suggestions for improving the business.   

## Sources and References:
<a href="https://mavenanalytics.io/project/893">Maven Market Dataset</a>

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

## Reading each file and cleanning data

In [28]:
# Read csv files
calendar = pd.read_csv("MavenMarket_Calendar.csv")
customer = pd.read_csv("MavenMarket_Customers.csv")
product = pd.read_csv("MavenMarket_Products.csv")
region = pd.read_csv("MavenMarket_Regions.csv")
store = pd.read_csv("MavenMarket_Stores.csv")
returns = pd.read_csv("MavenMarket_Returns_1997-1998.csv")


trans_1997 = pd.read_csv("MavenMarket Transactions/MavenMarket_Transactions_1997.csv")
trans_1998 = pd.read_csv("MavenMarket Transactions/MavenMarket_Transactions_1998.csv")
# Combine the 2 year transaction files
transactions = pd.concat([trans_1997, trans_1998])

### Customer 

In [3]:
customer.head()

,customer_id,customer_acct_num,first_name,last_name,customer_address,customer_city,customer_state_province,customer_postal_code,customer_country,birthdate,marital_status,yearly_income,gender,total_children,num_children_at_home,education,acct_open_date,member_card,occupation,homeowner
0,1,87462024688,Sheri,Nowmer,2433 Bailey Road,Tlaxiaco,Oaxaca,15057,Mexico,8/26/1961,M,$30K - $50K,F,4,2,Partial High School,9/10/1991,Bronze,Skilled Manual,Y
1,2,87470586299,Derrick,Whelply,2219 Dewing Avenue,Sooke,BC,17172,Canada,7/3/1915,S,$70K - $90K,M,1,0,Partial High School,3/11/1993,Bronze,Professional,N
2,3,87475757600,Jeanne,Derry,7640 First Ave.,Issaquah,WA,73980,USA,6/21/1910,M,$50K - $70K,F,1,1,Bachelors Degree,6/11/1991,Bronze,Professional,Y
3,4,87500482201,Michael,Spence,337 Tosca Way,Burnaby,BC,74674,Canada,6/20/1969,M,$10K - $30K,M,4,4,Partial High School,5/21/1994,Normal,Skilled Manual,N
4,5,87514054179,Maya,Gutierrez,8668 Via Neruda,Novato,CA,57355,USA,5/10/1951,S,$30K - $50K,F,3,0,Partial College,8/21/1992,Silver,Manual,N


In [4]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10281 entries, 0 to 10280
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   customer_id              10281 non-null  int64 
 1   customer_acct_num        10281 non-null  int64 
 2   first_name               10281 non-null  object
 3   last_name                10281 non-null  object
 4   customer_address         10281 non-null  object
 5   customer_city            10281 non-null  object
 6   customer_state_province  10281 non-null  object
 7   customer_postal_code     10281 non-null  int64 
 8   customer_country         10281 non-null  object
 9   birthdate                10281 non-null  object
 10  marital_status           10281 non-null  object
 11  yearly_income            10281 non-null  object
 12  gender                   10281 non-null  object
 13  total_children           10281 non-null  int64 
 14  num_children_at_home     10281 non-nul

In [5]:
# Change datatype for 4 columns below
customer["customer_acct_num"] = customer["customer_acct_num"].astype("object")
customer["customer_postal_code"] = customer["customer_postal_code"].astype("object")
customer["birthdate"] = pd.to_datetime(customer["birthdate"])
customer["acct_open_date"] = pd.to_datetime(customer["acct_open_date"])

In [6]:
# Create new column "full name"
customer["full_name"] = customer["first_name"] + " " + customer["last_name"]

In [9]:
# Create new column "current_age"
ref_date = dt.date(1998, 1, 1)
customer["current_age"] = customer["birthdate"].apply(lambda x: ref_date.year - int(str(x)[0:4]))

In [11]:
# Create new column "age_group"
def age_group(age):
    if age <= 18:
        return "Under 18"
    elif age <= 25:
        return "18-25"
    elif age <= 35:
        return "26-35"
    elif age <= 45:
        return "36-45"
    elif age <= 55:
        return "46-55"
    elif age <= 65:
        return "56-65"
    else:
        return "Over 65"

customer["age_group"] = customer["current_age"].apply(age_group)

In [12]:
customer.head()

,customer_id,customer_acct_num,first_name,last_name,customer_address,customer_city,customer_state_province,customer_postal_code,customer_country,birthdate,...,total_children,num_children_at_home,education,acct_open_date,member_card,occupation,homeowner,full_name,current_age,age_group
0,1,87462024688,Sheri,Nowmer,2433 Bailey Road,Tlaxiaco,Oaxaca,15057,Mexico,1961-08-26,...,4,2,Partial High School,1991-09-10,Bronze,Skilled Manual,Y,Sheri Nowmer,37,36-45
1,2,87470586299,Derrick,Whelply,2219 Dewing Avenue,Sooke,BC,17172,Canada,1915-07-03,...,1,0,Partial High School,1993-03-11,Bronze,Professional,N,Derrick Whelply,83,Over 65
2,3,87475757600,Jeanne,Derry,7640 First Ave.,Issaquah,WA,73980,USA,1910-06-21,...,1,1,Bachelors Degree,1991-06-11,Bronze,Professional,Y,Jeanne Derry,88,Over 65
3,4,87500482201,Michael,Spence,337 Tosca Way,Burnaby,BC,74674,Canada,1969-06-20,...,4,4,Partial High School,1994-05-21,Normal,Skilled Manual,N,Michael Spence,29,26-35
4,5,87514054179,Maya,Gutierrez,8668 Via Neruda,Novato,CA,57355,USA,1951-05-10,...,3,0,Partial College,1992-08-21,Silver,Manual,N,Maya Gutierrez,47,46-55


In [13]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10281 entries, 0 to 10280
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   customer_id              10281 non-null  int64         
 1   customer_acct_num        10281 non-null  object        
 2   first_name               10281 non-null  object        
 3   last_name                10281 non-null  object        
 4   customer_address         10281 non-null  object        
 5   customer_city            10281 non-null  object        
 6   customer_state_province  10281 non-null  object        
 7   customer_postal_code     10281 non-null  object        
 8   customer_country         10281 non-null  object        
 9   birthdate                10281 non-null  datetime64[ns]
 10  marital_status           10281 non-null  object        
 11  yearly_income            10281 non-null  object        
 12  gender                   10281 n

### Product 

In [14]:
product.head()

,product_id,product_brand,product_name,product_sku,product_retail_price,product_cost,product_weight,recyclable,low_fat
0,1,Washington,Washington Berry Juice,90748583674,2.85,0.94,8.39,NaN,NaN
1,2,Washington,Washington Mango Drink,96516502499,0.74,0.26,7.42,NaN,1.0
2,3,Washington,Washington Strawberry Drink,58427771925,0.83,0.40,13.10,1.0,1.0
3,4,Washington,Washington Cream Soda,64412155747,3.64,1.64,10.60,1.0,NaN
4,5,Washington,Washington Diet Soda,85561191439,2.19,0.77,6.66,1.0,NaN


In [15]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_id            1560 non-null   int64  
 1   product_brand         1560 non-null   object 
 2   product_name          1560 non-null   object 
 3   product_sku           1560 non-null   int64  
 4   product_retail_price  1560 non-null   float64
 5   product_cost          1560 non-null   float64
 6   product_weight        1560 non-null   float64
 7   recyclable            873 non-null    float64
 8   low_fat               552 non-null    float64
dtypes: float64(5), int64(2), object(2)
memory usage: 109.8+ KB


In [16]:
# Change datatype of product_sku to object
product["product_sku"] = product["product_sku"].astype("object")

In [17]:
product["recyclable"]

0       NaN
1       NaN
2       1.0
3       1.0
4       1.0
       ... 
1555    1.0
1556    1.0
1557    NaN
1558    NaN
1559    NaN
Name: recyclable, Length: 1560, dtype: float64

In [18]:
product['low_fat']

0       NaN
1       1.0
2       1.0
3       NaN
4       NaN
       ... 
1555    1.0
1556    NaN
1557    1.0
1558    NaN
1559    1.0
Name: low_fat, Length: 1560, dtype: float64

=> It seems that NaN values should be replaced with 0.

In [19]:
# We need to replace NaN with 0 in 2 above columns
product["recyclable"].fillna(0, inplace=True)
product["low_fat"].fillna(0, inplace=True)

In [20]:
# Check the info again
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_id            1560 non-null   int64  
 1   product_brand         1560 non-null   object 
 2   product_name          1560 non-null   object 
 3   product_sku           1560 non-null   object 
 4   product_retail_price  1560 non-null   float64
 5   product_cost          1560 non-null   float64
 6   product_weight        1560 non-null   float64
 7   recyclable            1560 non-null   float64
 8   low_fat               1560 non-null   float64
dtypes: float64(5), int64(1), object(3)
memory usage: 109.8+ KB


=> Everything is good. Let's move on to the next file. <br>

### Store 

In [21]:
store.head()

,store_id,region_id,store_type,store_name,store_street_address,store_city,store_state,store_country,store_phone,first_opened_date,last_remodel_date,total_sqft,grocery_sqft
0,1,28,Supermarket,Store 1,2853 Bailey Rd,Acapulco,Guerrero,Mexico,262-555-5124,1/9/1982,12/5/1990,23593,17475
1,2,78,Small Grocery,Store 2,5203 Catanzaro Way,Bellingham,WA,USA,605-555-8203,4/2/1970,6/4/1973,28206,22271
2,3,76,Supermarket,Store 3,1501 Ramsey Circle,Bremerton,WA,USA,509-555-1596,6/14/1959,11/19/1967,39696,24390
3,4,27,Gourmet Supermarket,Store 4,433 St George Dr,Camacho,Zacatecas,Mexico,304-555-1474,9/27/1994,12/1/1995,23759,16844
4,5,4,Small Grocery,Store 5,1250 Coggins Drive,Guadalajara,Jalisco,Mexico,801-555-4324,9/18/1978,6/29/1991,24597,15012


In [22]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   store_id              24 non-null     int64 
 1   region_id             24 non-null     int64 
 2   store_type            24 non-null     object
 3   store_name            24 non-null     object
 4   store_street_address  24 non-null     object
 5   store_city            24 non-null     object
 6   store_state           24 non-null     object
 7   store_country         24 non-null     object
 8   store_phone           24 non-null     object
 9   first_opened_date     24 non-null     object
 10  last_remodel_date     24 non-null     object
 11  total_sqft            24 non-null     int64 
 12  grocery_sqft          24 non-null     int64 
dtypes: int64(4), object(9)
memory usage: 2.6+ KB


In [23]:
# Change datatype to datetime for 2 below columns
store["first_opened_date"] = pd.to_datetime(store["first_opened_date"])
store["last_remodel_date"] = pd.to_datetime(store["last_remodel_date"])

### Region 

In [24]:
region.head()

,region_id,sales_district,sales_region
0,1,San Francisco,Central West
1,2,Mexico City,Mexico Central
2,3,Los Angeles,South West
3,4,Guadalajara,Mexico West
4,5,Vancouver,Canada West


In [25]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   region_id       109 non-null    int64 
 1   sales_district  109 non-null    object
 2   sales_region    109 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.7+ KB


=> Everything looks good.

### Calendar 

In [27]:
calendar.head()

,date
0,1/1/1997
1,1/2/1997
2,1/3/1997
3,1/4/1997
4,1/5/1997


In [29]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    730 non-null    object
dtypes: object(1)
memory usage: 5.8+ KB


In [30]:
# Change the datatype to datetime
calendar["date"] = pd.to_datetime(calendar["date"])

In [31]:
# Tạo các cột mới
week_start = 6  # 6 = Sunday, 0 = Monday
calendar["start_of_week"] = calendar["date"] - pd.to_timedelta((calendar["date"].dt.dayofweek - week_start) % 7, unit='d')
calendar["day_name"] = calendar["date"].dt.day_name()
calendar["month_name"] = calendar["date"].dt.month_name()
calendar["quarter"] = calendar["date"].dt.quarter
calendar["year"] = calendar["date"].dt.year


In [32]:
calendar

,date,start_of_week,day_name,month_name,quarter,year
0,1997-01-01,1996-12-29,Wednesday,January,1,1997
1,1997-01-02,1996-12-29,Thursday,January,1,1997
2,1997-01-03,1996-12-29,Friday,January,1,1997
3,1997-01-04,1996-12-29,Saturday,January,1,1997
4,1997-01-05,1997-01-05,Sunday,January,1,1997
...,...,...,...,...,...,...
725,1998-12-27,1998-12-27,Sunday,December,4,1998
726,1998-12-28,1998-12-27,Monday,December,4,1998
727,1998-12-29,1998-12-27,Tuesday,December,4,1998
728,1998-12-30,1998-12-27,Wednesday,December,4,1998


### Returns 

In [33]:
returns.head()

,return_date,product_id,store_id,quantity
0,1/1/1997,250,6,1
1,1/1/1997,628,6,1
2,1/1/1997,869,6,1
3,1/2/1997,469,11,1
4,1/2/1997,532,23,2


In [35]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7087 entries, 0 to 7086
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   return_date  7087 non-null   object
 1   product_id   7087 non-null   int64 
 2   store_id     7087 non-null   int64 
 3   quantity     7087 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 221.6+ KB


In [36]:
# Change datatype of column "return_date"
returns["return_date"] = pd.to_datetime(returns["return_date"])

In [39]:
# Check info again 
returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7087 entries, 0 to 7086
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   return_date  7087 non-null   datetime64[ns]
 1   product_id   7087 non-null   int64         
 2   store_id     7087 non-null   int64         
 3   quantity     7087 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 221.6 KB


### Transaction 

In [37]:
transactions.head()

,transaction_date,stock_date,product_id,customer_id,store_id,quantity
0,1/1/1997,12/31/1996,869,3449,6,5
1,1/1/1997,12/31/1996,1472,3449,6,3
2,1/1/1997,12/28/1996,76,3449,6,4
3,1/1/1997,12/26/1996,320,3449,6,3
4,1/1/1997,12/25/1996,4,3449,6,4


In [38]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269720 entries, 0 to 182882
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   transaction_date  269720 non-null  object
 1   stock_date        269720 non-null  object
 2   product_id        269720 non-null  int64 
 3   customer_id       269720 non-null  int64 
 4   store_id          269720 non-null  int64 
 5   quantity          269720 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 14.4+ MB


In [41]:
# Change datatype to datetime for 2 columns below
transactions["stock_date"] = pd.to_datetime(transactions["stock_date"])
transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"])

In [42]:
# Check info again
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269720 entries, 0 to 182882
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_date  269720 non-null  datetime64[ns]
 1   stock_date        269720 non-null  datetime64[ns]
 2   product_id        269720 non-null  int64         
 3   customer_id       269720 non-null  int64         
 4   store_id          269720 non-null  int64         
 5   quantity          269720 non-null  int64         
dtypes: datetime64[ns](2), int64(4)
memory usage: 14.4 MB


#### All the files are ready to use for visualization and analysis. Now, let's export the files and use Power BI to do the next step!

In [47]:
customer.to_csv('customer_export.csv', index=False)
product.to_csv('product_export.csv', index=False)
store.to_csv('store_export.csv', index=False)
region.to_csv('region_export.csv', index=False)
calendar.to_csv('calendar_export.csv', index=False)
returns.to_csv('returns_export.csv', index=False)
transactions.to_csv('transactions_export.csv', index=False)